<h1><center>Bert Inference Notebook </center></h1>
<h4><center>Final Project W266</center></h4>



In [1]:
!tail -20 'ner_dataset.csv'

",",",",O
M.D,NNP,O
JA25,NNP,O
Attending:,NNP,O
SYDNEY,NNP,O
DUESTERHAUS,NNP,O
",",",",O
M.D,NNP,O
MG85,NNP,O
EQ681/3978,NNP,O
Batch:,NNP,O
37609,CD,O
Index,NNP,O
No,NNP,O
FHOW8875S8,NNP,O
D:,NNP,O
6/10,CD,O
T:,NNP,O
1/22,CD,O
[report_end],NN,O


### II.2. Getting Started<a id="start" />

We start with some imports.

In [2]:
!pip install seqeval[gpu]

In [8]:
import warnings 
warnings.filterwarnings('ignore')
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
#import tensorflow_hub as hub
from time import time
import io
import re
from csv import reader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, TimeDistributed

from sklearn.model_selection import train_test_split

from tqdm import tqdm, trange


from datetime import datetime
import os
import sys
import zipfile

from keras.preprocessing.sequence import pad_sequences

import keras
import re

import numpy as np

from keras.models import load_model, model_from_json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM, GRU, Bidirectional
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers.noise import AlphaDropout
from keras.callbacks import ModelCheckpoint

from keras.layers.advanced_activations import LeakyReLU


In [52]:
import re
import pickle
import numpy as np
from collections import defaultdict


class Utils(object):
    def build_char_2_id_dict(self, data_set_char, min_freq):
        char_freq = defaultdict(int)
        char_2_id_table = {}

        for char in [char for label, seq in data_set_char for char in seq]:
            char_freq[char] += 1

        id_counter = 1

        for k, v in [(k, v) for k, v in char_freq.items() if v >= min_freq]:
            char_2_id_table[k] = id_counter
            id_counter += 1

        return char_2_id_table

    def build_data_set(self, data_set_char, char_2_id_dict, window_size):

        data_set = []

        for label, char_sequence in data_set_char:
            ids = []

            if len(char_sequence) == 2 * window_size + 1:
                for char in char_sequence:
                    if char in char_2_id_dict:
                        ids.append(char_2_id_dict[char])
                    else:
                        ids.append(0)

                feature_vector = np.array([float(ids[i])
                                           for i in range(0, len(ids))], dtype=float)

                data_set.append((float(label), feature_vector))

        return data_set

    def build_data_set_char(self, t, window_size):
        data_set_char_eos = \
            [(1.0, t[m.start() - window_size:m.start()].replace("\n", " ") +
              t[m.start():m.start() + window_size + 1].replace("\n", " "))
             for m in re.finditer('[\.:?!;~][^\n]?[\n]', t)]

        data_set_char_neos = \
            [(0.0, t[m.start() - window_size:m.start()].replace("\n", " ") +
              t[m.start():m.start() + window_size + 1].replace("\n", " "))
             for m in re.finditer('[\.:?!;~][^\s]?[ ]+', t)]

        return data_set_char_eos + data_set_char_neos

    def build_potential_eos_list(self, t, window_size):
        PUNCT = '[\(\)\u0093\u0094`“”\"›〈⟨〈<‹»«‘’–\'``'']*'
        EOS = '([\.:?!;~])'

        eos_positions = [(m.start())
                         for m in re.finditer(r'([\.:?!;~])(\s+' + PUNCT + '|' +
                                              PUNCT + '\s+|[\s\n]+)', t)]
        potential_eos_position = []

        for eos_position in eos_positions:
            left_context = t[eos_position - (2 * window_size):eos_position]
            right_context = t[eos_position:eos_position + (3 * window_size)]

            cleaned_left_context = left_context
            cleaned_right_context = right_context

            cleaned_left_context = re.sub('\s+', ' ', cleaned_left_context)
            cleaned_right_context = re.sub('\s+', ' ', cleaned_right_context)

            potential_eos_position.append((eos_position,
                                          cleaned_left_context[-window_size:] + t[eos_position] +
                                          cleaned_right_context[1:window_size + 1]))

        return potential_eos_position




In [9]:
json_file = open('sentence_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("sentence_model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


In [10]:

# Define mddaximal length of input 'sentences' (post tokenization).
max_word = 40
max_length = 64

In [11]:

import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [12]:
MAX_LEN = 128
bs = 32

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [14]:
torch.cuda.get_device_name(0) 

'Tesla P100-PCIE-16GB'

In [15]:
VOCAB = '/root/biobert_v1.0_pubmed_pmc/vocab.txt'
MODEL = '/root/biobert_v1.0_pubmed_pmc'

In [16]:
device = torch.device('cpu')
model2 = torch.load('/root/biobert.bin', map_location=lambda storage, loc: storage)
model2 = model2.cpu().double()
model2.eval();

In [17]:
tokenizer = BertTokenizer.from_pretrained('/root/biobert_v1.0_pubmed_pmc', do_lower_case=False)

In [18]:
columns = ['tag', 'cat']

nerDistribution = pd.read_csv('ner_tags', sep = '\t', names = columns) 

In [19]:
nerDistribution.tail(20)

,tag,cat
0,B-do,0
1,B-du,1
2,B-f,2
3,B-m,3
4,B-mo,4
5,B-r,5
6,I-do,6
7,I-du,7
8,I-f,8
9,I-m,9


In [20]:
from pprint import pprint

class ConnectionEngine:
    def __init__(self, class_list):
        self.class_list = class_list
        self._connected = list()
        self._connected.append(self._build_empty_row())

    def _build_empty_row(self):
        row = {k : None for k in self.class_list}
        return row
    
    def add_item(self, class_name, value):
        current_item = self._connected[-1]
        if current_item[class_name] == None:
            current_item[class_name] = value
        else:
            self._connected.append(self._build_empty_row())
            current_item = self._connected[-1]
            current_item[class_name] = value
        return
    
    def get_connected(self):
        return self._connected
            
    def print_out(self):
        for item in self._connected:
            print(item)

In [21]:
def detokenize(sent, label):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    new_label = []
    for i, tok in enumerate(zip(sent,label)):
        if tok[0].startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[0][2:]
        else:
            new_sent.append(tok[0])
            new_label.append(tok[1])
    return new_sent

In [22]:
def detokenize_label(sent, label):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    new_label = []
    for i, tok in enumerate(zip(sent,label)):
        if tok[0].startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[0][2:]
        else:
            new_sent.append(tok[0])
            new_label.append(tok[1])
    return new_label

In [141]:
def detokenize_entities(sent, label):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    new_label = []
    for i, tok in enumerate(zip(sent,label)):
        if tok[1].startswith("nerX") and not tok[0].startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[0]
            new_label[len(new_label) - 1] = new_label[len(new_label) - 1] + ''
        elif tok[1].startswith("nerX") and tok[0].startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[0][2:]
            new_label[len(new_label) - 1] = new_label[len(new_label) - 1] + ''
        elif tok[0].startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[0][2:]
            new_label[len(new_label) - 1] = new_label[len(new_label) - 1] + ''
        else:
            new_sent.append(tok[0])
            new_label.append(tok[1])
    return list(new_sent),list(new_label)

In [24]:
def printer(sent, label, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent, label)
    print(" ".join(sent))

In [25]:
def printer_label(sent, label, should_detokenize=True):
    if should_detokenize:
        sent = detokenize_label(sent, label)
    print(" ".join(sent))

In [26]:
def printer_entities(sent, label, should_detokenize=True):
    sent = []
    label = []
    if should_detokenize:
        sent, label = detokenize_entities(sent, label)
    print(" ".join(sent))
    print(" ".join(label))

In [105]:
training_file='data/combined.txt'
window_size=6
batch_size = 4


In [147]:
util = Utils()


with open(training_file, mode='r', encoding='utf-8') as f:
    training_corpus = f.read()

data_set_char = util.build_data_set_char(
    training_corpus, window_size)
char_2_id_dict = util.build_char_2_id_dict(
    data_set_char, min_freq)

#with open('data/test.txt', mode='r', encoding='utf-8') as f:
#    t = f.read()
t = 'The patient was given Plavix ( clopidogrel ) 75 mg po daily. Advised ATENOLAL 50 mg po weekly for the patient'
eos_marker="</eos>"


potential_eos_list = util.build_potential_eos_list(t, window_size)

eos_counter = 0

for potential_eos in potential_eos_list:
    start, char_sequence = potential_eos

    data_set = util.build_data_set([(-1.0, char_sequence)],
                                        char_2_id_dict,
                                        window_size)

    if len(data_set) > 0:
        label, feature_vector = data_set[0]

        predicted = model.predict(
            feature_vector.reshape(
                1,
                2 * window_size + 1),
            batch_size=batch_size,
            verbose=0)

        if predicted[0][0] >= .4:
            t = t[:(eos_counter * len(eos_marker)) + start + 1] + \
                eos_marker + t[(eos_counter * len(eos_marker)) + start + 1:]
            eos_counter += 1

blocked_text = t[:] + eos_marker
sent_list = blocked_text.split('</eos>')

In [149]:
def inference(sentence):
    
    tokenized_text_stg = [tokenizer.tokenize(sent) for sent in sentence.split()]
    tokenized_text = [y for x in tokenized_text_stg for y in x]
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(tokenized_text)],
                          maxlen=64, dtype="long", truncating="post", padding="post")
    attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
    
    b_input_ids = torch.tensor(input_ids)
    b_attention_masks = torch.tensor(attention_masks)


    logits = model2(b_input_ids, token_type_ids=None,
                           attention_mask=b_attention_masks)
    logits = logits.detach().cpu().numpy()
    predictions = ([list(p) for p in np.argmax(logits, axis=2)])
    pred_tags = np.array([nerDistribution.loc[(nerDistribution['cat'] == p_i).idxmax()][0] for p in predictions for p_i in p])
    sent = []
    label = []
    sent, label = detokenize_entities(tokenized_text, list(pred_tags))
    return sent, label
    
    

In [150]:
for i, sentence in enumerate(sent_list):
    #print(sentence)
    print(inference(sentence))

(['The', 'patient', 'was', 'given', 'Plavix', '(', 'clopidogrel', ')', '75', 'mg', 'po', 'daily.'], ['O', 'O', 'O', 'O', 'B-m', 'I-m', 'I-m', 'I-m', 'B-do', 'I-do', 'B-mo', 'B-f'])
(['Advised', 'ATENOLAL', '50', 'mg', 'po', 'weekly', 'for', 'the', 'patient'], ['O', 'B-m', 'B-do', 'I-do', 'B-mo', 'B-f', 'O', 'O', 'O'])
([], [])
